# Training Convolutional Neural Networks (convnets or CNNs) with tf.keras and LSUV initilization.

LSUV initialization proposed by Dmytro Mishkin and Jiri Matas in the article [All you need is a good Init](https://arxiv.org/pdf/1511.06422.pdf) consists of the two steps. 
 - First, pre-initialize weights of each convolution or inner-product layer with orthonormal matrices. 
 - Second, proceed from the first to the final layer, normalizing the variance of the output of each layer to be equal to one.

Original implementation can be found at [ducha-aiki/LSUVinit](https://github.com/ducha-aiki/LSUVinit).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import numpy as np
import time
from lib import LSUVinitialize

In [3]:
tf.random.set_seed(42)

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

In [5]:
TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images, dtype=np.float32) / 255

# Convert the train images and add channels
train_images = train_images.reshape((TRAINING_SIZE, 28, 28, 1))

test_images = np.asarray(test_images, dtype=np.float32) / 255
# Convert the train images and add channels
test_images = test_images.reshape((TEST_SIZE, 28, 28, 1))

In [6]:
# How many categories we are predicting from (0-9)
LABEL_DIMENSIONS = 10

train_labels  = tf.keras.utils.to_categorical(train_labels, LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels, LABEL_DIMENSIONS)

# Cast the labels to floats, needed later
train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt',
    'Sneaker', 'Bag', 'Ankle boot'
]

In [7]:
model = tf.keras.Sequential()
# model.add(tf.keras.Input(shape=train_images.shape[1:]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


You can see above that the output of every convolutional layer is a 3D tensor of shape (`height`, `width`, `filters`). The width and height tend to get smaller as we go deeper into the network and the number of filters or channels increases from the input channel size of 1. 

The last part of the network for the classification task is similar to the other notebooks and consists of `Dense` layers which process 1D vectors. So we first need to `Flatten` our 3D outputs from the convolutional part to 1D and then add the `Dense` layers:

In [8]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(LABEL_DIMENSIONS, activation=tf.nn.softmax))

Training the network is again similar to all the previous notebooks:

In [9]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [10]:
BATCH_SIZE=128

# Because tf.data may work with potentially **large** collections of data
# we do not shuffle the entire dataset by default
# Instead, we maintain a buffer of SHUFFLE_SIZE elements
# and sample from there.
SHUFFLE_SIZE = 10000 

# Create the dataset
training_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
training_dataset = training_dataset.shuffle(SHUFFLE_SIZE)
training_dataset = training_dataset.batch(BATCH_SIZE).repeat()

validation_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_dataset = validation_dataset.shuffle(SHUFFLE_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE).repeat()

In [11]:
def svd_orthonormal(shape):
    # Orthonorm init code is taked from Lasagne
    # https://github.com/Lasagne/Lasagne/blob/master/lasagne/init.py
    if len(shape) < 2:
        raise RuntimeError("Only shapes of length 2 or more are supported.")
    flat_shape = (shape[0], np.prod(shape[1:]))
    a = np.random.standard_normal(flat_shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == flat_shape else v
    q = q.reshape(shape)
    return q

In [12]:
model = LSUVinitialize(model, train_images[:BATCH_SIZE,:,:,:])

Init Layer conv2d
0.06591233
Init Layer conv2d_1
0.18357301
Init Layer conv2d_2
0.2051178
Init Layer dense
0.62251633
dense_1 too small
LSUV: total layers initialized 4


In [13]:
EPOCHS = 14

model.fit(training_dataset, 
    validation_data=validation_dataset,
    steps_per_epoch = 1000 // 32,
    epochs=EPOCHS,
    validation_steps=1000 // 32)

Train for 31 steps, validate for 31 steps
Epoch 1/14
31/31 [==============================] - 4s 127ms/step - loss: 1.4361 - accuracy: 0.5479 - val_loss: 0.8141 - val_accuracy: 0.7026
Epoch 2/14
31/31 [==============================] - 3s 85ms/step - loss: 0.7314 - accuracy: 0.7273 - val_loss: 0.6603 - val_accuracy: 0.7576
Epoch 3/14
31/31 [==============================] - 3s 84ms/step - loss: 0.6382 - accuracy: 0.7689 - val_loss: 0.5787 - val_accuracy: 0.7923
Epoch 4/14
31/31 [==============================] - 3s 81ms/step - loss: 0.5715 - accuracy: 0.7918 - val_loss: 0.5693 - val_accuracy: 0.7886
Epoch 5/14
31/31 [==============================] - 2s 81ms/step - loss: 0.5260 - accuracy: 0.8082 - val_loss: 0.5614 - val_accuracy: 0.7954
Epoch 6/14
31/31 [==============================] - 3s 84ms/step - loss: 0.5113 - accuracy: 0.8087 - val_loss: 0.4834 - val_accuracy: 0.8296
Epoch 7/14
31/31 [==============================] - 2s 75ms/step - loss: 0.4559 - accuracy: 0.8387 - val_loss: 

Again to evaluate the model we need to check the accuracy on unseen or test data:

In [14]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print('\nTest Model   Loss: %.6f\tAccuracy: %.6f' % (test_loss, test_acc))


Test Model   Loss: 0.395358	Accuracy: 0.856500
